In [4]:
!pip install torch torchvision torchaudio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.8 MB/s eta 0:00:00


In [2]:
!pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 MB 6.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 5.7 MB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: sympy
    Found existing installation: sympy 1.11.1
    Uninstalling sympy-1.11.1:
      Successfully uninstalled sympy-1.11.1
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1


In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torchvision.transforms.functional as TF
import random


# Directory structure
data_dir = '/Users/niteshyadav/Lyme_Disease/Lyme Dataset'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

# 1. Dataset Preparation & Normalization
class LymeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label, class_dir in enumerate(['Negative', 'Positive']):
            class_path = os.path.join(root_dir, class_dir)
            for img_file in os.listdir(class_path):
                self.image_paths.append(os.path.join(class_path, img_file))
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = TF.to_tensor(TF.pil_to_tensor(self.image_paths[idx]))
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Normalization transform
normalize = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_data = LymeDataset(train_dir, transform=normalize)
val_data = LymeDataset(val_dir, transform=normalize)
test_data = LymeDataset(test_dir, transform=normalize)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 2. Feature Extraction Model (using a pretrained CNN)
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        base_model = models.resnet18(pretrained=True)
        self.feature_layers = nn.Sequential(*list(base_model.children())[:-1])  # Remove the last FC layer

    def forward(self, x):
        with torch.no_grad():
            features = self.feature_layers(x)
        return features.view(features.size(0), -1)

feature_extractor = FeatureExtractor()

def extract_features(dataloader, model):
    model.eval()
    features, labels = [], []
    with torch.no_grad():
        for images, lbls in dataloader:
            feats = model(images)
            features.append(feats.cpu().numpy())
            labels.extend(lbls.cpu().numpy())
    return np.vstack(features), np.array(labels)

train_features, train_labels = extract_features(train_loader, feature_extractor)
val_features, val_labels = extract_features(val_loader, feature_extractor)
test_features, test_labels = extract_features(test_loader, feature_extractor)

# 3. Feature Ranking with Attention (EAR-FS style with a simple attention layer)
class FeatureRankingModel(nn.Module):
    def __init__(self, feature_dim):
        super(FeatureRankingModel, self).__init__()
        self.fc1 = nn.Linear(feature_dim, 512)
        self.attention = nn.Parameter(torch.ones(feature_dim))  # Learnable attention weights
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        attention_scores = torch.sigmoid(self.attention)
        weighted_x = x * attention_scores
        x = torch.relu(self.fc1(weighted_x))
        return self.fc2(x), attention_scores

feature_dim = train_features.shape[1]
ranking_model = FeatureRankingModel(feature_dim)

# Training the feature ranking model
def train_feature_ranking(model, features, labels, num_epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    dataset = torch.utils.data.TensorDataset(torch.tensor(features, dtype=torch.float32), torch.tensor(labels))
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch_features, batch_labels in dataloader:
            optimizer.zero_grad()
            outputs, _ = model(batch_features)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader)}")

train_feature_ranking(ranking_model, train_features, train_labels)

# Extract and rank features by importance
_, attention_scores = ranking_model(torch.tensor(train_features, dtype=torch.float32))
attention_scores = attention_scores.detach().numpy()
feature_ranking = np.argsort(attention_scores)[::-1]  # Rank by attention score (descending)

# 4. Subset Generation - Select Top Features
top_k = 100  # Choose the number of top features to select based on cross-validation
selected_features = feature_ranking[:top_k]

# Filter top features from dataset
train_features_top = train_features[:, selected_features]
val_features_top = val_features[:, selected_features]
test_features_top = test_features[:, selected_features]

# 5. Classifier Training on Selected Features to Avoid Overfitting
classifier = nn.Sequential(
    nn.Linear(top_k, 64),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2)
)

# Training function
def train_classifier(model, train_features, train_labels, val_features, val_labels, num_epochs=20):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(torch.tensor(train_features, dtype=torch.float32))
        loss = criterion(outputs, torch.tensor(train_labels))
        loss.backward()
        optimizer.step()

        # Validation Accuracy
        model.eval()
        with torch.no_grad():
            val_outputs = model(torch.tensor(val_features, dtype=torch.float32))
            _, val_preds = torch.max(val_outputs, 1)
            val_acc = accuracy_score(val_labels, val_preds.numpy())
            if val_acc > best_acc:
                best_acc = val_acc
                best_model_wts = model.state_dict().copy()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Acc: {val_acc}")

    # Load best weights
    model.load_state_dict(best_model_wts)
    return model

# Train and evaluate the model
classifier = train_classifier(classifier, train_features_top, train_labels, val_features_top, val_labels)

# Test the final model on test data
classifier.eval()
with torch.no_grad():
    test_outputs = classifier(torch.tensor(test_features_top, dtype=torch.float32))
    _, test_preds = torch.max(test_outputs, 1)
    test_accuracy = accuracy_score(test_labels, test_preds.numpy())

print("Test Accuracy on selected features:", test_accuracy)


ImportError: cannot import name 'get_ctx' from 'torch.library' (/Users/niteshyadav/anaconda3/lib/python3.10/site-packages/torch/library.py)

In [6]:
!pip uninstall torch torchvision torchaudio




Found existing installation: torch 2.5.0
Uninstalling torch-2.5.0:
  Would remove:
    /Users/niteshyadav/anaconda3/bin/convert-caffe2-to-onnx
    /Users/niteshyadav/anaconda3/bin/convert-onnx-to-caffe2
    /Users/niteshyadav/anaconda3/bin/torchfrtrace
    /Users/niteshyadav/anaconda3/bin/torchrun
    /Users/niteshyadav/anaconda3/lib/python3.10/site-packages/functorch/*
    /Users/niteshyadav/anaconda3/lib/python3.10/site-packages/torch-2.5.0.dist-info/*
    /Users/niteshyadav/anaconda3/lib/python3.10/site-packages/torch/*
    /Users/niteshyadav/anaconda3/lib/python3.10/site-packages/torchgen/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [7]:
!pip uninstall torchvision

Found existing installation: torchvision 0.20.0
Uninstalling torchvision-0.20.0:
  Would remove:
    /Users/niteshyadav/anaconda3/lib/python3.10/site-packages/torchvision-0.20.0.dist-info/*
    /Users/niteshyadav/anaconda3/lib/python3.10/site-packages/torchvision/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [8]:
!pip install torch==2.0.1+cpu torchvision==0.15.2+cpu torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu
ERROR: Could not find a version that satisfies the requirement torch==2.0.1+cpu (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0)
ERROR: No matching distribution found for torch==2.0.1+cpu


In [9]:
import torch
import torchvision
print("Torch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)


ImportError: cannot import name 'get_ctx' from 'torch.library' (/Users/niteshyadav/anaconda3/lib/python3.10/site-packages/torch/library.py)